In [1]:
import boto3
import numpy as np
import io
import pickle
import pymysql
import sys
import os

In [2]:
# Let's use Amazon S3
s3 = boto3.resource('s3')

In [3]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

amazon-braket-c5c33f4595cd
cf-templates-1eqlum4iklsii-us-east-2
lukinsivdatabucket


## Option 1: Upload and download pickled data

In [4]:
# Upload some data
s3_client = boto3.client('s3')
data = np.zeros(10)

In [5]:
# upload without using disk
# from https://stackoverflow.com/questions/48049557/how-to-write-npy-file-to-s3-directly
my_array_data = io.BytesIO()
pickle.dump(data, my_array_data)
my_array_data.seek(0)
s3_client.upload_fileobj(my_array_data, 'lukinsivdatabucket', 'your-file.pkl')


In [6]:
# download data
# download without using disk
my_array_data2 = io.BytesIO()
s3_client.download_fileobj('lukinsivdatabucket', 'your-file.pkl', my_array_data2)
my_array_data2.seek(0)
my_array2 = pickle.load(my_array_data2)

# check that everything is correct
np.allclose(data, my_array2)

True

## Option 2: Upload and download file

In [7]:
#alternativly: save file

data_txt = np.savetxt("testdata.txt", data)
data_open = open("testdata.txt", 'rb')


In [8]:
s3.Bucket('lukinsivdatabucket').put_object(Key='testdata.txt', Body=data_open)

s3.Object(bucket_name='lukinsivdatabucket', key='testdata.txt')

In [9]:
# retrieve it by manually downloading it
s3_client.download_file('lukinsivdatabucket', 'testdata.txt', 'downloaded_testdata.txt')

## Connecting to RDS

In [1]:
from decouple import config
from sqlalchemy import create_engine
from sqlalchemy import URL


In [2]:
# Retrieve value from .env file

HOST= config("HOST")
PORT= config("PORT")
DBNAME= config("DBNAME")
USER= config("USER")
PASSWORD= config("PASSWORD")

url_object = URL.create(
    "mysql+pymysql",
    username=USER,
    password=PASSWORD,  # plain (unescaped) text
    host=HOST,
    database=DBNAME,
)


In [3]:
engine = create_engine(url_object)

In [4]:
connection = engine.connect()

In [8]:
# setup database

from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]] =  mapped_column(String(30))

    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str] =  mapped_column(String(30))
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))

    user: Mapped["User"] = relationship(back_populates="addresses")

    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [9]:
Base.metadata.create_all(engine)


In [10]:
from sqlalchemy.orm import Session

with Session(engine) as session:

    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")

    session.add_all([spongebob, sandy, patrick])

    session.commit()


In [11]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))

for user in session.scalars(stmt):
    print(user)


User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')
